# Examples from READEME

In [ ]:
import numpy as np
def func(X,Y):
    return np.sin(X) * np.cos(2*Y)
a = np.linspace(0, np.pi, 27,endpoint=True)
b = np.linspace(0, np.pi, 28,endpoint=True)
fc = np.meshgrid(a, b, indexing='ij')
data = func(*fc)


In [ ]:
from pyrho.core.pgrid import PGrid
from pyrho.vis.plotly import get_plotly_scatter_plot
pg = PGrid(grid_data=data, lattice=[[np.pi,0], [0,np.pi]])
get_plotly_scatter_plot(pg.grid_data, pg.lattice, skips=1, opacity=1, marker_size=15)


In [ ]:
pg_2x = pg.get_transformed_obj([[1,1], [1,-1]], frac_shift=[0,0], grid_out=[30,32])
print(f"After doubling, rorating the (max, min) of the data changed from ({pg.grid_data.max():0.2f}, {pg.grid_data.min():0.2f}) to ({pg_2x.grid_data.max():0.2f}, {pg_2x.grid_data.min():0.2f})")
get_plotly_scatter_plot(pg_2x.grid_data, pg_2x.lattice, skips=1, opacity=1, marker_size=10)


In [ ]:
pg_2x = pg.get_transformed_obj([[1,1], [1,-1]], frac_shift=[0.5, 0.5], grid_out=[30,32])
print(f"After doubling, rorating and shifting the (max, min) of the data changed from ({pg.grid_data.max():0.2f}, {pg.grid_data.min():0.2f}) to ({pg_2x.grid_data.max():0.2f}, {pg_2x.grid_data.min():0.2f})")
get_plotly_scatter_plot(pg_2x.grid_data, pg_2x.lattice, skips=1, opacity=1, marker_size=10)


In [ ]:
from pyrho.core.chargeDensity import ChargeDensity
from pymatgen.io.vasp import Chgcar
from pyrho.core.pgrid import PGrid
from pyrho.core.utils import get_padded_array
from pyrho.core.fourier import PFourier
from pyrho.vis.plotly import get_plotly_scatter_plot
chgcar = Chgcar.from_hdf5("../test_files/Si.uc.hdf5")
chgcar = ChargeDensity.from_pmg_volumetric_data(chgcar)
chgcar.reorient_axis()


# Get the 1-D plot 

In [ ]:
pg1 = PGrid(chgcar.grid_data[10,10,:], lattice=[chgcar.lattice[0,0]])
get_plotly_scatter_plot(pg1.grid_data, lat_mat=pg1.lattice, skips=1)

# 2-D Plot

In [ ]:
pg2 = PGrid(chgcar.grid_data[24,:,:], chgcar.lattice[:2,:2])
get_plotly_scatter_plot(pg2.grid_data, lat_mat=pg2.lattice, skips=4)

# 3-D Plot

In [ ]:
chgcar.grid_data.shape

In [ ]:
get_plotly_scatter_plot(chgcar.grid_data, lat_mat=chgcar.lattice, skips=4, mask=chgcar.grid_data > 0.5, marker_size=10)

In [ ]:
chgcar_x2 = chgcar.get_transformed_obj(sc_mat = [[1,1,0],[1,-1,0],[0,0,1]], frac_shift=[0.5,0.5,0.5], grid_out=[120,120,60])
get_plotly_scatter_plot(chgcar_x2.grid_data, lat_mat=chgcar_x2.lattice, skips=4, mask=chgcar_x2.grid_data > 0.5, marker_size=10)

In [ ]:
chgcar_x2.grid_data.shape

In [ ]:
ndim = len(data_in.shape)
if ndim > 3:
    raise NotImplementedError("Can only render data of 1, 2, or 3 dimensions.")

ss = slice(0, None, 4)
trimmed_data = np.real(data_in).copy()
all_slices = (ss, ) * ndim
trimmed_data = trimmed_data[all_slices]
trimmed_data.shape

In [ ]:
def show_2d(pg_2D):
    av=np.linspace(0,1,pg_2D.grid_data.shape[0],endpoint=False)
    bv=np.linspace(0,1,pg_2D.grid_data.shape[1],endpoint=False)
    AA, BB = np.meshgrid(av,bv,indexing='ij') #indexing to match the lablled array
    xx, yy = np.dot(pg_2D.lattice.T[:2,:2], [AA.flatten(),BB.flatten()])
    xshift, yshift = np.dot(pg_2D.lattice.T[:2,:2], ((av[1]-av[0])/2.,(bv[1]-bv[0])/2.))
    plt.scatter(xx+xshift, yy+yshift, c=np.log(pg_2D.grid_data.flatten()), edgecolors='black',alpha=0.1)
show_2d(pg2)


In [ ]:
fft_data = np.abs(np.fft.fftn(pg_2D.grid_data))
pf = PFourier(fft_data, pg_2D.lattice)

def show_2d_f(pf, thresh = 12):
    av=np.linspace(0,1,pf.fourier_data.shape[0],endpoint=False)
    bv=np.linspace(0,1,pf.fourier_data.shape[1],endpoint=False)

    xx, yy = pf.fft_pos_centered_cartesian_s
    mask = pf.fourier_data.flatten() > thresh
    plt.scatter(xx[mask], yy[mask], c=pf.fourier_data.flatten()[mask], edgecolors='black',alpha=0.4)
    plt.axes().set_aspect("equal")


In [ ]:
fft_data = np.abs(np.fft.fftn(pg_2D.grid_data))
pf = PFourier(fft_data, pg_2D.lattice)
show_2d_f(pf)

In [ ]:
g1,g2 = pg_2D.grid_data.shape
pg_super = pg_2D.get_transformed_obj(
    sc_mat=[[1, 1], [1, -2]], frac_shift=[0, 0], grid_out=[g1*3 , g2*2 ], up_sample=2
)
show_2d(pg_super)


In [ ]:
fft_data_super = np.abs(np.fft.fftn(pg_super.grid_data))
pf_super = PFourier(fft_data_super, pg_super.lattice)


In [ ]:
pg_2D.lattice

In [ ]:
show_2d_f(pf_super, thresh = 3*9)

In [ ]:
show_2d_f(pf, thresh = 3)